# Data Preparation

In this step we read in the data set from different .csv files, have a look inot the data and merge them into on pandas dataframe.

In [2]:
# we will work with pandas
import pandas as pd

## Bakery sales data set

In [3]:
# load the sales dataset
sales_data = pd.read_csv('../bakery_sales_data/umsatzdaten_gekuerzt.csv')

# change date format before merge to have same datatype on key
sales_data['Datum'] = pd.to_datetime(sales_data['Datum'])

# show some sample information to get an idea of the content
print(sales_data.head(3))
print("\n")
print(sales_data.tail(3))
print("\n")
print(sales_data.sample(5))
print("\n")
sales_data.info()


       Datum  Warengruppe      Umsatz
0 2013-07-01            1  148.828353
1 2013-07-02            1  159.793757
2 2013-07-03            1  111.885594


          Datum  Warengruppe     Umsatz
9331 2017-12-23            6  84.062223
9332 2017-12-24            6  60.981969
9333 2017-12-27            6  34.972644


          Datum  Warengruppe      Umsatz
2585 2015-08-20            2  504.081148
892  2015-12-24            1  195.399801
9318 2017-12-10            6   94.149766
4701 2016-06-17            3  111.578949
1183 2016-10-19            1  110.217733


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9334 entries, 0 to 9333
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Datum        9334 non-null   datetime64[ns]
 1   Warengruppe  9334 non-null   int64         
 2   Umsatz       9334 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 218.9 KB


### Analysis of the content
The data are sales figures, presumably in euros, broken down by product group and per day.

## Weather data

In [4]:
# load the weather dataset
weather_data = pd.read_csv('../bakery_sales_data/wetter.csv')

# change date format before merge to have same datatype on key
weather_data['Datum'] = pd.to_datetime(weather_data['Datum'])

# calculate and add mean temperature per month and deviation per day
# extract month and year
weather_data['Jahr_Monat'] = weather_data['Datum'].dt.to_period('M')

# calulate mean temperature per month
monthly_mean_temp = weather_data.groupby('Jahr_Monat')['Temperatur'].transform('mean')

# add the mean temperature as extra colum
weather_data['Durchschnittstemperatur'] = monthly_mean_temp

weather_data['Temperature_Abweichung'] = weather_data['Temperatur'] - weather_data['Durchschnittstemperatur']


# show some sample information to get an idea of the content
print(weather_data.head(3))
print("\n")
print(weather_data.tail(3))
print("\n")
print(weather_data.sample(5))
print("\n")
weather_data.info()


       Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0 2012-01-01         8.0      9.8250                   14        58.0   
1 2012-01-02         7.0      7.4375                   12         NaN   
2 2012-01-03         8.0      5.5375                   18        63.0   

  Jahr_Monat  Durchschnittstemperatur  Temperature_Abweichung  
0    2012-01                 3.395276                6.429724  
1    2012-01                 3.395276                4.042224  
2    2012-01                 3.395276                2.142224  


          Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
2598 2019-07-30         7.0     20.7375                    8        61.0   
2599 2019-07-31         6.0     20.4500                    7        61.0   
2600 2019-08-01         5.0     21.0625                    9        61.0   

     Jahr_Monat  Durchschnittstemperatur  Temperature_Abweichung  
2598    2019-07                19.852823                0.884677  
2599    20

### Analysis of the content
Each entry in the data represents weather observations for a specific day at a specific location.

# Kieler Woche data

In [5]:
# load the kiwo dataset
kiwo_data = pd.read_csv('../bakery_sales_data/kiwo.csv')

# change date format before merge to have same datatype on key
kiwo_data['Datum'] = pd.to_datetime(kiwo_data['Datum'])

# show some sample information to get an idea of the content
print(kiwo_data.head(3))
print("\n")
print(kiwo_data.tail(3))
print("\n")
print(kiwo_data.sample(5))
print("\n")
kiwo_data.info()

       Datum  KielerWoche
0 2012-06-16            1
1 2012-06-17            1
2 2012-06-18            1


        Datum  KielerWoche
69 2019-06-28            1
70 2019-06-29            1
71 2019-06-30            1


        Datum  KielerWoche
40 2016-06-22            1
3  2012-06-19            1
11 2013-06-24            1
50 2017-06-22            1
55 2018-06-17            1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Datum        72 non-null     datetime64[ns]
 1   KielerWoche  72 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.2 KB


## Additional data binary (0/1)
- Feiertage 
- Wahltage
- Schulferien
- Wochentage

In [6]:
# load datasets
holyday_data = pd.read_csv('../bakery_sales_data/feiertage_sh.csv')
schhol_data = pd.read_csv('../bakery_sales_data/schulferien.csv')
elec_data = pd.read_csv('../bakery_sales_data/wahltage.csv')

# change data type of Datum with specified format
holyday_data['Datum'] = pd.to_datetime(holyday_data['Datum'], format='%d.%m.%Y')
schhol_data['Datum'] = pd.to_datetime(schhol_data['Datum'])
elec_data['Datum'] = pd.to_datetime(elec_data['Datum'], format='%m/%d/%Y')

# start and end date for data frame
start_date = '2012-01-01'
end_date = '2019-08-01'

# pre-data frame
full_time_series = pd.date_range(start=start_date, end=end_date, freq='D')
pre_data = pd.DataFrame(full_time_series, columns=['Datum'])
pre_data = pre_data.merge(holyday_data, on='Datum', how='left')\
    .merge(schhol_data, on='Datum', how='left')\
    .merge(elec_data, on='Datum', how='left')\

# dictionary holyday code
feiertage_dict = {
    'Neujahr': 1,
    'Karfreitag': 2,
    'Ostermontag': 3,
    'Maifeiertag': 4,
    'Christi Himmelfahrt': 5,
    'Pfingstmontag': 6,
    'Tag der Deutschen Einheit': 7,
    '1. Weihnachtstag': 8,
    '2. Weihnachtstag': 9
}

# dictionary school holyday code
holyday_dict = {
    'Sommerferien': 1,
    'Keine Ferien': 0,
    'Herbstferien': 2,
    'Weihnachtsferien': 3,
    'Osterferien': 4,
    'Pfingsten': 0
}

# mapping-function
def map_holyday_to_code(feiertag):
    return feiertage_dict.get(feiertag, 0)  

# mapping-function
def map_schoolhol_to_code(ferientag):
    return holyday_dict.get(ferientag, 0)  

# HolyCode/Schhol = numeric code, BinHoly = binary holydays 0 = no, 1 = yes
pre_data['HolyCode'] = pre_data['Feiertag'].apply(map_holyday_to_code)
pre_data['SchholCode'] = pre_data['Ferien'].apply(map_schoolhol_to_code)
pre_data['BinHoly'] = (pre_data['HolyCode'] != 0).astype(int)
pre_data['BinSchhol'] = (pre_data['SchholCode'] != 0).astype(int)
pre_data['BinElec'] = pre_data['Wahl'].apply(lambda x: 0 if pd.isnull(x) else 1)
# delete 
pre_data.drop(columns=['Feiertag', 'Ferien', 'Wahl'], inplace=True)

# add day of week
pre_data['DayOfWeek'] = pre_data['Datum'].dt.day_name()
# dictonary
weekend_dict = {
    'Saturday': 1,
    'Sunday': 1,
    'Monday': 0,
    'Tuesday': 0,
    'Wednesday': 0,
    'Thursday': 0,
    'Freiday': 0
}
# function 
def map_days_to_code(wdays):
    return weekend_dict.get(wdays, 0) 

# new row weekend
pre_data['weekend'] = pre_data['DayOfWeek'].apply(map_days_to_code)

print(pre_data)

          Datum  HolyCode  SchholCode  BinHoly  BinSchhol  BinElec  DayOfWeek  \
0    2012-01-01         0           0        0          0        0     Sunday   
1    2012-01-02         0           0        0          0        0     Monday   
2    2012-01-03         0           0        0          0        0    Tuesday   
3    2012-01-04         0           0        0          0        0  Wednesday   
4    2012-01-05         0           0        0          0        0   Thursday   
...         ...       ...         ...      ...        ...      ...        ...   
2765 2019-07-28         0           0        0          0        0     Sunday   
2766 2019-07-29         0           0        0          0        0     Monday   
2767 2019-07-30         0           0        0          0        0    Tuesday   
2768 2019-07-31         0           0        0          0        0  Wednesday   
2769 2019-08-01         0           0        0          0        0   Thursday   

      weekend  
0          

### Analysis of the content
The dates list the days on which the "Kieler Woche", a major local event, took place.

## Merge of the three datasets
We will use sales data as base table and left join wetherdata and kiwo data together.

### Data preparation for merging
wetter.csv: start = 2012-01-01, end = 2019-08-01
kiwo.csv:   start = 2012-06-16, end = 2019-06-30
umsatz.csv: start = 2013-07-01. end = 2017-12-27

In [7]:
# start and end date for data frame
start_date = '2013-07-01'
end_date = '2019-08-01'

# pre-data frame to set date range
full_time_series = pd.date_range(start=start_date, end=end_date, freq='D')
bakery_sales_prediction_data = pd.DataFrame(full_time_series, columns=['Datum'])

# left join of non-binary data and filling missing data with NaN
bakery_sales_prediction_data = sales_data.merge(weather_data, on='Datum', how='left')
bakery_sales_prediction_data.fillna('NaN', inplace=True)

# left join of binary data for kiwo and other events and filling missing with 0 = no, 1 = yes
bakery_sales_prediction_data = bakery_sales_prediction_data.merge(kiwo_data, on='Datum', how='left')\
    .merge(pre_data, on='Datum', how='left')
bakery_sales_prediction_data.fillna(0, inplace=True)
bakery_sales_prediction_data['KielerWoche'] = bakery_sales_prediction_data['KielerWoche'].astype(int)

# show sample data
print(bakery_sales_prediction_data.head(3))
print("\n")
print(bakery_sales_prediction_data.tail(3))
print("\n")
print(bakery_sales_prediction_data.sample(5))
print("\n")
bakery_sales_prediction_data.info()

       Datum  Warengruppe      Umsatz Bewoelkung Temperatur  \
0 2013-07-01            1  148.828353        6.0    17.8375   
1 2013-07-02            1  159.793757        3.0    17.3125   
2 2013-07-03            1  111.885594        7.0     21.075   

  Windgeschwindigkeit Wettercode Jahr_Monat Durchschnittstemperatur  \
0                15.0       20.0    2013-07                21.90121   
1                10.0        NaN    2013-07                21.90121   
2                 6.0       61.0    2013-07                21.90121   

  Temperature_Abweichung  KielerWoche  HolyCode  SchholCode  BinHoly  \
0               -4.06371            0         0           1        0   
1               -4.58871            0         0           1        0   
2               -0.82621            0         0           1        0   

   BinSchhol  BinElec  DayOfWeek  weekend  
0          1        0     Monday        0  
1          1        0    Tuesday        0  
2          1        0  Wednesday        0

/tmp/ipykernel_18202/2656741510.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NaN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  bakery_sales_prediction_data.fillna('NaN', inplace=True)
/tmp/ipykernel_18202/2656741510.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with period[M], please explicitly cast to a compatible dtype first.
  bakery_sales_prediction_data.fillna(0, inplace=True)


Save bakery_sales_prediction_data as Pickle File to preserve datatypes and could later easy import file as input for the models

In [8]:
# set output path
output_path = '../bakery_sales_data/bakery_sales_prediction_data.pkl'

# print as csv
bakery_sales_prediction_data.to_pickle(output_path)
print(f'DataFrame saved to {output_path}')


DataFrame saved to ../bakery_sales_data/bakery_sales_prediction_data.pkl
